In [13]:
import pickle
import librosa
import numpy as np
import os

SLICE_LENGTH = 5 # seconds

## Feature Extraction

In [2]:
def extract_features(y, sr):
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    rmse = librosa.feature.rms(y=y)[0]
    
    features = np.hstack((np.mean(mfccs, axis=1), 
                          np.mean(spectral_centroid), 
                          np.mean(spectral_rolloff), 
                          np.mean(zcr), 
                          np.mean(rmse)))
    return features


## Main
#### Load Model


In [21]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)


#### Predict
Input complete file path tp this method to return the forecast as a list  
forecast = `[chest_pain,bp,palpitations,other_disease]`

In [22]:

def predict(file_path):
    data, sr = librosa.load(file_path)

    audio_slices = [data[i:(i+(sr*SLICE_LENGTH))] for i in range(0, len(data), sr*SLICE_LENGTH)]

    x_audio = [extract_features(audio_slice , sr) for audio_slice in audio_slices]
    x_data = np.array([[22, 0]] * len(x_audio))
    x = np.concatenate((x_data, x_audio), axis=1)

    preds = model.predict(x)

    # Reduce predictions taking mean 
    mean_pred = np.mean(preds, axis=0) 
    forecast = list(np.round(mean_pred, 0).astype(int))

    return forecast

# int(any(item == 1 for item in forecast))
